In [6]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
import string
import re

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lmuns\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
Prices_with_changes = pd.read_csv('../DataFiles/withPricesAndChange.csv')


Prices_with_changes=  Prices_with_changes.round(4)
Prices_with_changes.loc[(Prices_with_changes.StockChg == 'Positive'), 'StockChg'] = 1
Prices_with_changes.loc[(Prices_with_changes.StockChg == 'Negative'), 'StockChg'] = 0
Prices_with_changes["StockChg"] = pd.to_numeric(Prices_with_changes["StockChg"])

Prices_with_changes

,title,Symbol,Security,Year,Month,Day,Date,Key,Opening,Closing,StockChg,WeekNo
0,Agilent to Buy Multiplicom for ‚Ç¨68M in Cash,A,Agilent Technologies,2016,12,20,2016-12-20,2016-12-20A,45.75,46.21,1,52
1,Pressure Biosciences CEO On Its Beaten Down St...,A,Agilent Technologies,2016,12,6,2016-12-06,2016-12-06A,44.58,44.84,1,50
2,"The Market In 5 Minutes: DryShips, Trump And T...",A,Agilent Technologies,2016,11,16,2016-11-16,2016-11-16A,46.30,46.18,0,47
3,A Peek Into The Markets: U.S. Stock Futures De...,A,Agilent Technologies,2016,11,16,2016-11-16,2016-11-16A,46.30,46.18,0,47
4,"Keep an Eye on These 8 Stocks for November 16,...",A,Agilent Technologies,2016,11,16,2016-11-16,2016-11-16A,46.30,46.18,0,47
...,...,...,...,...,...,...,...,...,...,...,...,...
176894,Mid-Morning Market Update: Unemployment Reache...,ZTS,Zoetis,2013,2,1,2013-02-01,2013-02-01ZTS,31.50,31.01,0,5
176895,Pfizer Offers Statement on Zoetis IPO,ZTS,Zoetis,2013,2,1,2013-02-01,2013-02-01ZTS,31.50,31.01,0,5
176896,"Zoetis IPO Opens at $31.50/Share, Priced at $26",ZTS,Zoetis,2013,2,1,2013-02-01,2013-02-01ZTS,31.50,31.01,0,5
176897,Zoetis IPO to Open Shortly,ZTS,Zoetis,2013,2,1,2013-02-01,2013-02-01ZTS,31.50,31.01,0,5


In [46]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=15000)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()

big_ol_text_blob = Prices_with_changes['title']

big_ol_text_blob = big_ol_text_blob.tolist()
big_ol_text_blob = big_ol_text_blob
#big_ol_text_blob = ''.join(big_ol_text_blob)

#big_ol_text_blob = [big_ol_text_blob]

clean_blobs = []



for blob in big_ol_text_blob:
    blob = blob.lower()
    words = blob.split()
    new_blob = ""
    for word in words:
        if not word in set(stopwords.words('english')):
            new_blob = new_blob + " " + word + " "
        
    nres = re.sub(r'[^\w\s]', '', new_blob)
    clean_blobs.append(nres)

clean_blobs



[' agilent  buy  multiplicom  ç68m  cash ',
 ' pressure  biosciences  ceo  beaten  stock  we  begin  focus  area  important ',
 ' market  5  minutes  dryships  trump  target ',
 ' peek  markets  us  stock  futures  decline  ahead  economic  data ',
 ' keep  eye  8  stocks  november  16  2016 ',
 ' agilent  technologies  sees  fy17  rev  435b437b  vs  est  435b  eps  210216  vs  est  219  q4  rev  104104b  vs  est  108b  eps  048050  vs  est  053 ',
 ' agilent  technologies  reports  q4  eps  059  vs  est  052  rev  111b  vs  est  107b ',
 ' 8  stocks  watching  today ',
 ' earnings  scheduled  november  15  2016 ',
 ' leerink  swann  assumes  agilent  technologies  inc  common  stock  outperform ',
 ' goldmans  7  favorite  biotech  ideas  2016 ',
 ' goldman  bullish  agilent  technologies  raises  price  target  53 ',
 ' btig  thinks  agilents  transformation  early  innings ',
 ' 10  biggest  price  target  changes  thursday ',
 ' goldman  keeps  agilent  conviction  buy  list  follo

In [ ]:
clean_blobs

In [42]:
counts = cv.fit_transform(big_ol_text_blob)
counts

<176899x1500 sparse matrix of type '<class 'numpy.int64'>'
	with 1388578 stored elements in Compressed Sparse Row format>

In [47]:
from sklearn.feature_extraction.text import TfidfTransformer
cv = CountVectorizer()
tfidf_transformer = TfidfTransformer()
X = counts
y = np.array(Prices_with_changes['StockChg'])

In [48]:
from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(X.toarray(), y, test_size=0.2, random_state = 0)

In [49]:
#LD model

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
model = LinearDiscriminantAnalysis().fit(x_train, y_train)

predictions = model.predict(x_test)
predictions

from sklearn.metrics import classification_report
print( classification_report(y_test, predictions) )

              precision    recall  f1-score   support

           0       0.52      0.42      0.46     16942
           1       0.55      0.65      0.59     18438

    accuracy                           0.54     35380
   macro avg       0.53      0.53      0.53     35380
weighted avg       0.53      0.54      0.53     35380



In [51]:
#SVG MODEL
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(x_train, y_train)
print('Test Acc: %.3f' % model.score(x_test, y_test))

from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))


In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(x_train, y_train)
predictions = gnb.predict(x_test)
print(classification_report(y_test, predictions,))


In [ ]:
#use stddev to determine if the movement is statictically signimifant. need historic stock data for the day each of these various articles was written. 